In [6]:
import requests
from bs4 import BeautifulSoup as bs

In [22]:
r = requests.get("https://www.kufar.by/l?ot=1&query=кольцо&rgn=all")
print(r.status_code)

200


In [23]:
info = bs(r.text, "html.parser")
print(info)

<!DOCTYPE html>
<html lang="ru"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>«Кольцо»‎ на Куфаре</title><link href="https://www.kufar.by/l?query=Кольцо" rel="canonical"/><meta content="Смотрите, что я нашел: «Кольцо» купить на Куфаре – крупнейшей площадке объявлений в Беларуси" name="description"/><meta content="https://content.kufar.by/static/frontend/img/kufar_logo_meta.png" name="image"/><meta content="Куфар - крупнейшая площадка объявлений в Беларуси" property="og:site_name"/><meta content="website" property="og:type"/><meta content="«Кольцо»‎ на Куфаре" property="og:title"/><meta content="https://www.kufar.by/l?query=Кольцо" property="og:url"/><meta content="Смотрите, что я нашел: «Кольцо» купить на Куфаре – крупнейшей площадке объявлений в Беларуси" property="og:description"/><meta content="https://content.kufar.by/static/frontend/img/kufar_logo_meta.png" property="og:image"/><meta content="image/png" property="og:image:type"/><meta cont

In [24]:
ring_names = info.find_all('a', class_='styles_wrapper__5FoK7')
print(ring_names)

[<a class="styles_wrapper__5FoK7" href="https://auto.kufar.by/vi/149386180?searchId=907e167aa42acc1c970a1e5f539347b617ba" target="_blank"><div class="styles_image__kwQfE"><div class="styles_container__MBp_G"><div class="styles_image__7aRPM"><div class="styles_container__mneN4" style="padding-top:100%"><div class="styles_image___d77M"><div class="styles_container__Un__X"><img alt="Колёсные гайки, болты, центровочные кольца" class="styles_image__ZPJzx lazyload" data-src="https://rms.kufar.by/v1/list_thumbs_2x/adim1/3362a983-ee05-4e1f-af12-bebb4b6a9365.jpg" loading="lazy"/></div></div></div></div><object><a class="styles_polepos__583hs" href="https://www.kufar.by/account/promotion_services?serviceType=vip" rel="noopener noreferrer" target="_blank"><img alt="vip" src="https://content.kufar.by/static/frontend/svg/paid-services-card/v2/vip.svg"/></a></object><div class="styles_border__MT38b"></div></div></div><div class="styles_content__5GOlX styles_content--highlighted__oaGLA"><div class="s

In [26]:
print(len(ring_names))
for name in ring_names:
    print(name["href"])

54
https://auto.kufar.by/vi/149386180?searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/168451275?searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/131314872?searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/192046589?rank=0&searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/203938131?rank=1&searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/161579354?rank=2&searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/160847900?rank=3&searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/184197603?rank=4&searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/203935970?rank=5&searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/203933982?rank=6&searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/112331468?rank=7&searchId=907e167aa42acc1c970a1e5f539347b617ba
https://www.kufar.by/item/203932510?rank

In [30]:
all_requests = []
for elem in ring_names:
    h = requests.get(elem["href"])
    all_requests.append(h)

In [32]:
print(all_requests[1].text)

<!DOCTYPE html><html lang="ru"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><title>Обручальное Кольцо*Распродажа По цене прошлого год, цена 19.99 р. купить в Минске на Куфаре - Объявление №168451275</title><link rel="canonical" href="https://www.kufar.by/item/168451275"/><meta name="robots" content="all"/><meta name="description" content="➜  Обратите внимание на объявление: Обручальное Кольцо*Распродажа По цене прошлого год, цена 19.99 р.. Состояние: Новое. Покупайте и продавайте товары на Kufar - крупнейшей площадке объявлений в Минске."/><meta property="og:site_name" content="Куфар - крупнейшая площадка объявлений в Беларуси"/><meta property="og:type" content="website"/><meta property="og:title" content="Обручальное Кольцо*Распродажа По цене прошлого год, цена 19.99 р. купить в Минске на Куфаре - Объявление №168451275"/><meta property="og:url" content="https://www.kufar.by/item/168451275"/><meta property="og:description" content="➜  Обратите вниман

In [34]:
request_html = [bs(elem.text, "html.parser") for elem in all_requests]

In [49]:
all_descriptions = [elem.find_all('div', itemprop = "description") for elem in request_html]

In [50]:
print(all_descriptions)

[[<div class="styles_description_content__raCHR" itemprop="description">Новые колёсные гайки, болты, центровочные кольца, декоративные колпачки на гайки разных размеров на все машины, возможна отправка европочтой, пишите на какую машину или размер который нужен, цену уточняйте.<br/></div>], [<div itemprop="description">Обручальное Кольцо *Распродажа По цене прошлого года!* ширина 3 мм -19,99 р. ширина 5 мм -24,99 р.<br/></div>], [<div itemprop="description">Кольца канализационные разных размеров :0.7м,1.0м,1.5м,2.0м. Армированные сеткой(ячейка 200/200/4). Предоставление услуги-доставка манипулятором. <br/></div>], [<div itemprop="description">Обручальные кольца 70-х годов новые. Золото (проба 375, 583). Продаются как изделия. По одному не продаются. <br/></div>], [<div itemprop="description">Продам серебряное кольцо состояние хорошее, проба 925, звоните <br/></div>], [<div itemprop="description">Монтаж канализации под ключ:<br/>Монтаж канализации при наличии высоких грунтовых вод;<br/>

In [68]:
good_indexes = []
for num, elem in enumerate(all_descriptions):
    if ("золот" in elem[0].text or "Золот" in elem[0].text) and "озолот" not in elem[0].text:
        good_indexes.append(num)
print(good_indexes)

[3, 8, 11, 14, 15, 16, 18, 23, 27, 33, 41]


In [67]:
with open("test.txt", "w") as file:
    for index in good_indexes:
        file.write(ring_names[index]["href"] + '\n')